In [15]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
import os


In [17]:
key = os.getenv("TAVILY_API_KEY")
# print(key)

# TAVIRY を使用した RAG の実装

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from azure.identity import ClientSecretCredential, get_bearer_token_provider
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template('''
                                          以下の文脈だけを踏まえて質問に回答してください
                                          文脈:"""
                                          {context}
                                          """
                                          質問: {question}
                                          ''')

cred = ClientSecretCredential(
    client_id=os.getenv("client_id"),
    client_secret=os.getenv("client_secret"),
    tenant_id=os.getenv("tenant_id")
)

token_provider = get_bearer_token_provider(
    cred,
    "https://cognitiveservices.azure.com/.default"
)

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-02-15-preview",
    azure_endpoint=os.getenv("azure_endpoint"),
    azure_ad_token_provider=token_provider
)

In [18]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

In [19]:
from langchain_core.runnables import RunnablePassthrough

# assign method を使うことで、 ans に結果を格納でき、retriever からの応答を別に使えます
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | RunnablePassthrough.assign( ans = prompt | llm | StrOutputParser())
)

In [22]:
output = chain.invoke("東京の明日の天気は？")
print(output)
print(f"ans: {output['ans']}")

{'context': [Document(metadata={'title': '東京の明日の天気予報：東京新聞デジタル', 'source': 'https://www.tokyo-np.co.jp/weather/area_tomorrow/0313', 'score': 0.89279664, 'images': []}, page_content='* 天気トップ * 全国の天気 * 雨雲レーダー * 注意報･警報 * 台風 * 地震 * 津波 * 紫外線 * 花粉 今日の天気 明日の天気 8月30日（土）11時00分発表 最高気温 36℃ 最低気温 27℃ 晴れ 時々 くもり 所により 昼過ぎ から 雨 で 雷を伴う 降水確率 確率 時刻 洗濯指数 とてもよく乾く 花粉 少ない 紫外線 強い * 時刻 * 予報 * 気温(℃) * 風向 + 南 + 南 + 南 + 南 + 北西 + 北西 + 北西 + 東 * 風速 + 疾風 + 疾風 + 疾風 + 弱風 + 弱風 + 弱風 + 弱風 + 弱風 ※時刻の過ぎた部分は過去の予報値です。 ## 週間天気 8月30日（土）11時00分発表 東京地方 雷    伊豆諸島北部 解除    伊豆諸島南部 解除    小笠原諸島 雷    ２３区西部\u3000発表注意報・警報はなし   ２３区東部\u3000発表注意報・警報はなし   多摩北部\u3000発表注意報・警報はなし   多摩西部\u3000(継続)雷注意報   多摩南部\u3000発表注意報・警報はなし   大島\u3000(解除)濃霧注意報   新島\u3000(解除)濃霧注意報   八丈島\u3000発表注意報・警報はなし   三宅島\u3000(解除)濃霧注意報   小笠原諸島\u3000(継続)雷注意報'), Document(metadata={'title': '東京の天気予報 - ウェザーニュース', 'source': 'https://weathernews.jp/onebox/tenki/tokyo/', 'score': 0.7734393, 'images': []}, page_content='* トップ * 関東 * 東京 # 東京の天気予報 28木 29金 30土 31日 1月 2火 3水 4木 雲が多めでも厳しい残暑 金